# `CLOUD.jl` - 2D linear advection example

In [74]:
using CLOUD, OrdinaryDiffEq

Define the physical problem

In [75]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)))
initial_data = InitialDataSine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

Set discretization parameters

In [76]:
M = 4
p = 4
p_map = 4
form = WeakConservationForm()
strategy = Lazy()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.1*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [77]:
reference_approximation =ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGQuadrature(),
    facet_quadrature_rule=LGQuadrature(),
    mapping_degree=p_map, N_plot=50)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.2)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

Run the solver

In [78]:
ode_problem_1 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy);

save_solution(ode_problem_1.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem_1, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:          124μs / 526689.8%        11.8KiB / 5375653.0%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual          40.8k    457ms   69.6%  11.2μs    293MiB   47.1%  7.34KiB
   volume terms         40.8k    266ms   40.5%  6.51μs    146MiB   23.5%  3.66KiB
   facet terms          40.8k   97.8ms   14.9%  2.40μs   96.5MiB   15.5%  2.42KiB
   mass matrix solve    40.8k   44.1ms    6.7%  1.08μs   19.9MiB    3.2%     512B
 extrap solution        40.8k   63.0ms    9.6%  1.54μs   56.1MiB    9.0%  1.41KiB
 eval num flux          40.8k   53.0ms    8.1%  1.30μs    105MiB   16.9%  2.62KiB
 gather ext sta

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [79]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.0057491190465866005
Conservation (initial/final/diff):
[8.023589626732219e-8][8.023589448749591e-8][-1.779826286352204e-15]
Energy (initial/final/diff):
[0.12499874137052301][0.12439699898414946][-0.0006017423863735466]


Write output to VTK file

In [80]:
postprocess_vtk(spatial_discretization, string(results_path,"solution.vtu"), last(sol.u));